In [1]:
from logging import getLogger
from recbole.utils import init_logger, init_seed
from recbole.trainer import Trainer
from custom.doctr import Doctr
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
import pickle

import torch

In [ ]:
config = Config(model=Doctr, dataset='trial_zero', config_file_list=["./atomic/doctr.yaml"])
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()

logger.info(config)

# dataset filtering
dataset = create_dataset(config)
#dataset = pickle.load(open('./dataset.pkl', 'rb'))
logger.info(dataset)
print('-------------')

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

30 Apr 23:58    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = D:\OneDrive - University of Edinburgh\paper\[01 PRE] Trial Recomm\\atomic\trial_zero
checkpoint_dir = D:\OneDrive - University of Edinburgh\paper\[01 PRE] Trial Recomm\\models\
show_progress = True
save_dataset = True
dataset_save_path = D:\OneDrive - University of Edinburgh\paper\[01 PRE] Trial Recomm\models\trial_zero-dataset.pth
save_dataloaders = True
dataloaders_save_path = D:\OneDrive - University of Edinburgh\paper\[01 PRE] Trial Recomm\models\trial_zero-for-Doctr-dataloader.pth
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'uniform': 1, 'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Para

-------------


30 Apr 23:58    INFO  Load split dataloaders from: [D:\OneDrive - University of Edinburgh\paper\[01 PRE] Trial Recomm\models\trial_zero-for-Doctr-dataloader.pth]
30 Apr 23:58    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'uniform': 1, 'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
30 Apr 23:58    INFO  [Evaluation]: eval_batch_size = [2048] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


In [3]:
model = Doctr(config, train_data.dataset).to(config['device'])
trainer = Trainer(config, model)
trainer.eval_collector.data_collect(train_data)
trainer.eval_collector.model_collect(model)

In [4]:
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

30 Apr 23:58    INFO  epoch 0 training [time: 1.00s, train loss: 2.9381]
30 Apr 23:58    INFO  epoch 0 evaluating [time: 2.17s, valid_score: 0.670563]
30 Apr 23:58    INFO  valid result: 
hit@5 : 0.0024    hit@10 : 0.0024    hit@20 : 0.0024    composite_similarity@5 : 0.723370373249054    composite_similarity@10 : 0.6740361452102661    composite_similarity@20 : 0.6446613669395447    composite_similarity_gt : 0.6705628633499146    recall@5 : 0.0024    recall@10 : 0.0024    recall@20 : 0.0024    ndcg@5 : 0.0024    ndcg@10 : 0.0024    ndcg@20 : 0.0024
30 Apr 23:58    INFO  Saving current: D:\OneDrive - University of Edinburgh\paper\[01 PRE] Trial Recomm\\models\Doctr-Apr-30-2024_23-58-14.pth
30 Apr 23:58    INFO  epoch 1 training [time: 0.86s, train loss: 1.8078]
30 Apr 23:58    INFO  epoch 1 evaluating [time: 2.10s, valid_score: 0.699428]
30 Apr 23:58    INFO  valid result: 
hit@5 : 0.0071    hit@10 : 0.0141    hit@20 : 0.0259    composite_similarity@5 : 0.7310892939567566    composite_s

In [5]:
test_result = trainer.evaluate(test_data, load_best_model=True)

30 Apr 23:59    INFO  Loading model structure and parameters from D:\OneDrive - University of Edinburgh\paper\[01 PRE] Trial Recomm\\models\Doctr-Apr-30-2024_23-58-14.pth


In [3]:
test_result # Custom model, freeze=True

OrderedDict([('composite_similarity@5', 0.6022577881813049),
             ('composite_similarity@10', 0.5917636156082153),
             ('composite_similarity@20', 0.598173975944519),
             ('composite_similarity_gt', 0.5942211151123047)])